In [ ]:
import glob
import pandas as pd
import folium #run pip install folium in terminal
import pyarrow #run pip install pyarrow in terminal
import folium
from folium import plugins
from branca.element import Figure

In [ ]:
parent_directory = "/Users/rebecca/roads_and_cams/"
df_speedbands = pd.read_csv(parent_directory + "traffic_speedbands_no_dup.csv")
df_images = pd.read_csv(parent_directory + "traffic_images_no_dup.csv")
df_incidents = pd.read_csv(parent_directory + "traffic_incidents_no_dup.csv")

#latitude and longitude of SG
sg_lat = 1.3521
sg_lng = 103.8198

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns #run pip install seaborn in terminal

In [ ]:
df_speedbands[['lat1', 'long1','lat2','long2']] = df_speedbands['Location'].str.split(' ', expand=True)
df_speedbands.rename(columns={'Extract Time':'ExtractTime'}, inplace=True)
df_speedbands

In [ ]:
markers_one = df_speedbands[['lat1', 'long1']].to_numpy()
#markers_one

In [ ]:
#map of lat1 long1 speedbands

# create new figure
fig_one = Figure(height = 550, width = 750)

# create new map
map_one = folium.Map([sg_lat, sg_lng], zoom_start = 11,
                             min_zoom = 8, max_zoom = 16)
fig_one.add_child(map_one)

# create heatmap and add to map
plugins.HeatMap(markers_one, radius = 4, blur = 6).add_to(map_one)

map_one

In [ ]:
#df_images
markers_cam = df_images[['Latitude', 'Longitude']].to_numpy()
#markers_cam

In [ ]:
# create new figure
fig_two = Figure(height = 550, width = 750)

# create new map
map_two = folium.Map([sg_lat, sg_lng], zoom_start = 11,
                             min_zoom = 8, max_zoom = 16)
fig_two.add_child(map_two)

# create circles and add to map
for row in df_images.index:
    folium.CircleMarker(location=[df_images["Latitude"][row],df_images["Longitude"][row]],
                        radius=2,
                        weight=5, tooltip= df_images["CameraID"][row]).add_to(map_two)

map_two

In [ ]:
# function to get start and ending position of trajectory
def get_start_end_pos(trj):
  return (trj.iloc[0, :], trj.iloc[-1, :])

In [ ]:
# create new figure
fig_roads = Figure(height = 550, width = 750)

# add map to figure
m_roads = folium.Map(location = [sg_lat, sg_lng],
                tiles = None, zoom_start = 11,
                min_zoom = 11, max_zoom = 16)
fig_roads.add_child(m_roads)

folium.TileLayer('cartodbpositron', name='Road Categories').add_to(m_roads)

def route(row):
    route_start = row[["lat1", "long1"]].rename(columns={"lat1": "lat", "long1": "lng"}).astype(float)
    route_end = row[["lat2", "long2"]].rename(columns={"lat2": "lat", "long2": "lng"}).astype(float)
    route = pd.concat([route_start,route_end])
    return route

# Creating feature groups
for grp_name, df_grp in df_speedbands.groupby('RoadCategory'):
    feature_group = folium.FeatureGroup(grp_name)
    for row in df_grp.itertuples():
        row=pd.DataFrame.from_records([row], columns=row._fields)
        #print(row.RoadName[0])
        # Adding lines to the different feature groups
        line_1 = folium.vector_layers.PolyLine(route(row),
                                        popup = '<b>LinkID: </b>'+str(row.LinkID[0])+'<br><b>Road Category: </b>'+ row.RoadCategory[0] +'<br><b>Coordinates: </b>'+ row.Location[0],
                                        tooltip = row.RoadName[0],
                                        color = 'blue', weight = 1, opacity = 0.5).add_to(feature_group)          
        # add ploylines to map
        feature_group.add_to(m_roads)

        # get start and end points of each trajectory
        route_positions = get_start_end_pos(route(row))
        '''
        # add markers for trj 1
        folium.Marker(
            location = [route_positions[0][0], route_positions[0][1]],
            popup = "start point",
            icon = folium.Icon(color = "green"),
        ).add_to(feature_group)

        folium.Marker(
            location = [route_positions[1][0], route_positions[1][1]],
            popup = "start point",
            icon = folium.Icon(color = "red"),
        ).add_to(feature_group)
        '''

# add layer control to toggle layers
folium.LayerControl().add_to(m_roads)
fig_roads

In [ ]:
#both cams and roads

def route(row):
    route_start = row[["lat1", "long1"]].rename(columns={"lat1": "lat", "long1": "lng"}).astype(float)
    route_end = row[["lat2", "long2"]].rename(columns={"lat2": "lat", "long2": "lng"}).astype(float)
    route = pd.concat([route_start,route_end])
    return route

import base64

def cam_img(rowind):
    index = len("https://dm-traffic-camera-itsc.s3.ap-southeast-1.amazonaws.com/")
    folder = df_images['ImageLink'][rowind][index:index+len("2022-10-08/08-50")].replace("/","_").replace("-","_")
    index = len("https://dm-traffic-camera-itsc.s3.ap-southeast-1.amazonaws.com/2022-10-08/08-50/")
    filename = df_images['ImageLink'][rowind][index:index+len("1001_0843_20221008085001_525510.jpg")]
    encoded = base64.b64encode(open("./images/"+folder+"/"+filename, 'rb').read()).decode()
    html = '<img src="data:image/jpeg;base64,{}" img style="width:100%; height:100%;">'.format
    iframe = folium.IFrame(html(encoded), width=320, height=240)
    popup = folium.Popup(iframe, max_width=320)
    return popup

# create new figure
roads_and_cams_fig = Figure(height = 550, width = 750)

# add map to figure
roads_and_cams_map = folium.Map(location = [sg_lat, sg_lng],
                tiles = None, zoom_start = 11,
                min_zoom = 11, max_zoom = 16)
roads_and_cams_fig.add_child(roads_and_cams_map)


folium.TileLayer('cartodbpositron', name='Road Categories').add_to(roads_and_cams_map)

# Creating feature groups
for grp_name, df_grp in df_speedbands.groupby('RoadCategory'):
    feature_group = folium.FeatureGroup(grp_name)
    for row in df_grp.itertuples():
        row=pd.DataFrame.from_records([row], columns=row._fields)


        # Adding lines to the different feature groups
        line_1 = folium.vector_layers.PolyLine(route(row),
                                        popup = '<b>LinkID: </b>'+str(row.LinkID[0])+'<br><b>Road Category: </b>'+ row.RoadCategory[0] +'<br><b>Coordinates: </b>'+ row.Location[0],
                                        tooltip = row.RoadName[0],
                                        color = 'blue', weight = 2, opacity = 0.5).add_to(feature_group)          
        # add ploylines to map
        feature_group.add_to(roads_and_cams_map)


for rowind in df_images.index:
    folium.CircleMarker(location=[df_images["Latitude"].iloc[rowind],df_images["Longitude"].iloc[rowind]],
                        radius=2,
                        weight=5, tooltip= df_images["CameraID"][rowind], popup = cam_img(rowind), opacity = 0.8).add_to(roads_and_cams_map)

# add layer control to toggle layers
folium.LayerControl().add_to(roads_and_cams_map)

roads_and_cams_fig.save('roads_and_cams_fig.html')
roads_and_cams_fig

